In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
df = pd.read_csv('../DataSets/eurusd.csv', parse_dates = ['Date'], index_col = 'Date')

In [ ]:
df

In [ ]:
sma1 = 50
sma2 = 200

In [ ]:
df['sma_s'] = df['price'].rolling(window = sma1).mean()
df['sma_l'] = df['price'].rolling(window = sma2).mean()

In [ ]:
df

In [ ]:
df.plot(figsize = (12, 8), title = 'EUR/USD - SMA{} | SMA{}'.format(sma1, sma2), fontsize = 12)
plt.legend(fontsize = 12);

In [ ]:
df.dropna(inplace = True)

In [ ]:
df

In [ ]:
df.loc['2016'].plot(figsize = (12, 8), title = 'EUR/USD - SMA{} | SMA{}'.format(sma1, sma2), fontsize = 12)
plt.legend(fontsize = 12);

In [ ]:
df['position'] = np.where(df['sma_s'] > df['sma_l'], 1, -1)

In [ ]:
df

In [ ]:
df.loc[:, ['sma_s', 'sma_l', 'position']].plot(figsize = (12, 8), fontsize = 12, secondary_y = 'position', title = 'EUR/USD - SMA{} | SMA{}'.format(sma1, sma2));

In [ ]:
df.loc['2016', ['sma_s', 'sma_l', 'position']].plot(figsize = (12, 8), fontsize = 12, secondary_y = 'position', title = 'EUR/USD - SMA{} | SMA{}'.format(sma1, sma2));

# Vectorized Strategy Backtesting

In [ ]:
df

In [ ]:
df['returns'] = np.log(df['price'] / df['price'].shift(1))

In [ ]:
df

In [ ]:
df['strategy'] = df['position'].shift(1) * df['returns']

In [ ]:
df

In [ ]:
df.dropna(inplace = True)

In [ ]:
# calculates absolute performance
df[['returns', 'strategy']].sum()

In [ ]:
# calculates absolute performance
df[['returns', 'strategy']].sum().apply(np.exp)

In [ ]:
# annualized returns
df[['returns', 'strategy']].mean() * 252

In [ ]:
# annualized risk
df[['returns', 'strategy']].std() * np.sqrt(252)

In [ ]:
df['creturns'] = df['returns'].cumsum().apply(np.exp)
df['cstrategy'] = df['strategy'].cumsum().apply(np.exp)

In [ ]:
df

In [ ]:
df[['creturns', 'cstrategy']].plot(figsize = (12, 8), title = 'EUR/USD - SMA{} | SMA{}'.format(sma1, sma2))
plt.legend(fontsize = 12);

In [ ]:
outperf = df['cstrategy'].iloc[-1] - df['creturns'].iloc[-1]
outperf

# Optimal SMA Strategy

In [ ]:
data = pd.read_csv('../DataSets/eurusd.csv', parse_dates = ['Date'], index_col = 'Date')

In [ ]:
data

## Creates a function that runs a SMA strategy

In [ ]:
def run_strategy(SMA):
    df = data.copy()
    df['returns'] = np.log(df['price'] / df['price'].shift(1))
    df['sma_s'] = df['price'].rolling(window = int(SMA[0])).mean()
    df['sma_l'] = df['price'].rolling(window = int(SMA[1])).mean()
    df.dropna(inplace = True)
    
    df['position'] = np.where(df['sma_s'] > df['sma_l'], 1, -1)
    df['strategy'] = df['position'].shift(1) * df['returns']
    df.dropna(inplace = True)
    
    return df[['returns', 'strategy']].sum().apply(np.exp)

In [ ]:
run_strategy((50, 200))

In [ ]:
run_strategy((10, 50))

In [ ]:
run_strategy((22, 252))

## Finds the Optimal Windows for the SMA Strategy

In [ ]:
from itertools import product

In [ ]:
sma_1 = range(10, 50, 1)
sma_2 = range(100, 252, 1)

In [ ]:
results = pd.DataFrame()

for SMA_1, SMA_2 in product(sma_1, sma_2):
    df = data.copy()
    data.dropna(inplace = True)
    df['returns'] = np.log(df['price'] / df['price'].shift(1))
    df['sma_s'] = df['price'].rolling(window = int(SMA_1)).mean()
    df['sma_l'] = df['price'].rolling(window = int(SMA_2)).mean()
    df.dropna(inplace = True)
    
    df['position'] = np.where(df['sma_s'] > df['sma_l'], 1, -1)
    df['strategy'] = df['position'].shift(1) * df['returns']
    df.dropna(inplace = True)
    
    perf = df[['returns', 'strategy']].sum().apply(np.exp)
    
    results = results.append(pd.DataFrame({
        'SMA_1': SMA_1, 'SMA_2': SMA_2, 
        'MARKET' : perf['returns'], 
        'STRATEGY' : perf['strategy'], 
        'OUT' : perf['strategy'] - perf['returns']
    }, index = [0]), ignore_index = True)

In [ ]:
results

In [ ]:
results.info()

In [ ]:
results.sort_values('OUT', ascending = False).head(10)

## Optimal Strategy using Scipy!

In [ ]:
def run_strategy(SMA):
    df = data.copy()
    df['returns'] = np.log(df['price'] / df['price'].shift(1))
    df['sma_s'] = df['price'].rolling(window = int(SMA[0])).mean()
    df['sma_l'] = df['price'].rolling(window = int(SMA[1])).mean()
    df.dropna(inplace = True)
    
    df['position'] = np.where(df['sma_s'] > df['sma_l'], 1, -1)
    df['strategy'] = df['position'].shift(1) * df['returns']
    df.dropna(inplace = True)
    
    return -df[['returns', 'strategy']].sum().apply(np.exp)[-1]
# maximizes absolute performance

In [ ]:
from scipy.optimize import brute

In [ ]:
brute(run_strategy, ((10, 50, 1), (100, 252, 1)))

In [ ]:
-run_strategy((46, 137))

### Run strategy using our original function

In [ ]:
def run_strategy(SMA):
    df = data.copy()
    df['returns'] = np.log(df['price'] / df['price'].shift(1))
    df['sma_s'] = df['price'].rolling(window = int(SMA[0])).mean()
    df['sma_l'] = df['price'].rolling(window = int(SMA[1])).mean()
    df.dropna(inplace = True)
    
    df['position'] = np.where(df['sma_s'] > df['sma_l'], 1, -1)
    df['strategy'] = df['position'].shift(1) * df['returns']
    df.dropna(inplace = True)
    
    return df[['returns', 'strategy']].sum().apply(np.exp)

In [ ]:
run_strategy((46, 137))